In [43]:
import numpy
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec.load('nepa_doc2vec_model')
model.delete_temporary_training_data(keep_doctags_vectors=True,keep_inference = True)


import os.path
import gensim
import pandas as pd
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

index_dir = os.path.join('../../../scratch/boilerplate/big_text_files')

lda_results = pd.read_csv('../../../scratch/boilerplate/lda_combo_results_V2.csv')
lda_results = lda_results[lda_results['score']>=400]
lda_results = lda_results[lda_results['a_id']!=lda_results['b_id']]

lda_results['a_id'] = [str(i) for i in lda_results['a_id']]
lda_results['b_id'] = [str(i) for i in lda_results['b_id']]

a_is_eis = lda_results['a_id'].str.contains(r'^[0-9]{8}')
b_is_eis = lda_results['b_id'].str.contains(r'^[0-9]{8}')
lda_eis_results = lda_results[a_is_eis & b_is_eis]


doe_meta = pd.read_csv(os.path.join(index_dir, 'big_doe_metadata.txt'),header = None)
import re
doe_meta['PROJECT_ID'] = [re.sub(r'(--|_).*','',id) for id in doe_meta[0]]
usfs_meta = pd.read_csv(os.path.join(index_dir, 'big_usfs_metadata.txt'),header = None)
usfs_meta['PROJECT_ID'] = [re.sub(r'(--|_).*','',id) for id in usfs_meta[0]]
blm_meta = pd.read_csv(os.path.join(index_dir, 'big_blm_metadata.txt'),header = None)
blm_meta['PROJECT_ID'] = [re.sub(r'(--|_).*','',id) for id in blm_meta[0]]
eis_meta = pd.read_csv(os.path.join(index_dir, 'big_eis_metadata.txt'),header = None)
eis_meta['PROJECT_ID'] = [re.sub(r'(--|_).*','',id) for id in eis_meta[0]]
meta_corpus = blm_meta.append(doe_meta).append(eis_meta).append(usfs_meta)
meta_corpus.index = range(meta_corpus.shape[0])

meta_corpus['page'] = [str(s) for s in meta_corpus[1]]
meta_corpus['id'] = meta_corpus.PROJECT_ID +'--'+meta_corpus[0]+'--'+ meta_corpus.page
meta_corpus['id'] = meta_corpus['id'].str.replace(r"(.+-{2})\1",r"\1")

corp_id = [id for id in meta_corpus.id]
pages_in_eis400_lda = meta_corpus['id'].isin(lda_eis_results['a'])|meta_corpus['id'].isin(lda_eis_results['b'])

# Set file names for train and test data
data_dir = os.path.join('scratch/boilerplate/big_text_files')
doe_file = os.path.join(data_dir, 'big_doe_text_only.txt')
usfs_file = os.path.join(data_dir, 'big_usfs_text_only.txt')
blm_file = os.path.join(data_dir, 'big_blm_text_only.txt')
eis_file = os.path.join(data_dir, 'big_eis_text_only.txt')

import smart_open

def read_corpus(fname, tokens_only=False,lines_are_documents=True,tag = 'doc',meta = [],maxrow = float("inf")):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if i>= maxrow: break
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # version that makes id tag for page
                #yield gensim.models.doc2vec.TaggedDocument(tokens, [tag,meta['PROJECT_ID'][i],i])
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [tag,meta['PROJECT_ID'][i]])


blm_corpus = list(read_corpus(blm_file,tag = 'blm',meta = blm_meta))
doe_corpus = list(read_corpus(doe_file,tag = 'doe',meta = doe_meta))
eis_corpus = list(read_corpus(eis_file,tag = 'eis',meta = eis_meta))
usfs_corpus = list(read_corpus(usfs_file,tag = 'usfs',meta = usfs_meta))

train_corpus = blm_corpus + doe_corpus + eis_corpus + usfs_corpus 


In [104]:
pages_in_eis400_lda = meta_corpus['id'].isin(lda_eis_results['a'])|meta_corpus['id'].isin(lda_eis_results['b'])

In [106]:
eis400_ldacorpus = meta_corpus[pages_in_eis400_lda]

TypeError: unhashable type: 'list'

In [85]:
test in corpus_id_list

True

In [48]:
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer
import re
from nltk.corpus import stopwords
import pandas as pd

In [47]:
align_file=pd.read_csv('../../../scratch/boilerplate/lda_combo_results_V2.csv')

In [49]:
import os
import pandas as pd
data_dir = os.path.join('../../../scratch/boilerplate/big_text_files')
#lee_train_file = os.path.join(test_data_dir, 'big_doe_text_only.txt')
doe_meta = pd.read_csv(os.path.join(data_dir, 'big_doe_metadata.txt'),header = None)
usfs_meta = pd.read_csv(os.path.join(data_dir, 'big_usfs_metadata.txt'),header = None)
blm_meta = pd.read_csv(os.path.join(data_dir, 'big_blm_metadata.txt'),header = None)
eis_meta = pd.read_csv(os.path.join(data_dir, 'big_eis_metadata.txt'),header = None)

In [5]:
meta_corpus = eis_meta.append(blm_meta).append(doe_meta).append(usfs_meta)

In [38]:

doe_corpus = list(read_corpus(doe_file,tag = 'doe'))
eis_corpus = list(read_corpus(eis_file))
usfs_corpus = list(read_corpus(usfs_file))
blm_corpus = list(read_corpus(blm_file))
#train_corpus = doe_corpus + eis_corpus + usfs_corpus + blm_corpus

In [ ]:
import multiprocessing
print(f"My machine has {multiprocessing.cpu_count()} cores.")
from multiprocessing.pool import Pool
pool = Pool()
#appending all the vectors in a list for training


In [116]:
X = pool.map(model.infer_vector,[e[0] for e in train_corpus])


KeyboardInterrupt: 

In [ ]:
pool.close()

In [ ]:
X=[]
for i in range(len(model.docvecs)):
    X.append(model.docvecs[i])
    #print mdoel.docvecs[i]
dvecs = []
for d in range(len(train_corpus)):
    dvec = model.infer_vector(train_corpus[d][0])
    dvecs.append(dvec)

In [68]:
#import the modules
from sklearn.cluster import KMeans
import numpy as np
#create the kmeans object withe vectors created previously
kmeans = KMeans(n_clusters=40, random_state=0).fit(X)


In [ ]:

#print all the labels
print kmeans.labels_
#create a dictionary to get cluster data
clusters={0:[],1:[],2:[],3:[]}
for i in range(40):
    clusters[kmeans.labels_[i]].append(' '.join(df_new.ix[i,'title_l']))
print clusters

In [82]:
#import the modules
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(df_new['title_l']))]


NameError: name 'df_new' is not defined

In [101]:
def preprocess(str):
    # remove links
    str = re.sub(r'http(s)?:\/\/\S*? ', "", str)
    return str


def preprocess_document(text):
    text = preprocess(text)
    return ''.join([x if x.isalnum() or x.isspace() else " " for x in text ]).split()

In [67]:




# computes cluster Id for document vectors
doc_ids,_ = vq(model.docvecs,centroids)

# zips cluster Ids back to document labels 
doc_labels = zip(model.docvecs.doctags.keys(), doc_ids)


ValueError: object arrays are not supported

KeyboardInterrupt: 

In [36]:
NUM_CLUSTERS = 20
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)


AttributeError: 'Word2VecKeyedVectors' object has no attribute 'shape'

In [7]:
#model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=4, epochs=40,seed = 24)

In [8]:
#model.build_vocab(train_corpus)

In [9]:
#model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [10]:

# saving a model trained via Gensim's fastText implementation
#model.save('nepa_doc2vec_model')
#loaded_model = FT_gensim.load('saved_model_gensim')
#print(loaded_model)

# saving a model trained via fastText wrapper
#model_wrapper.save('saved_model_wrapper')
#loaded_model = FT_wrapper.load('saved_model_wrapper')
#print(loaded_model)

In [78]:
#vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
#print(vector)

[ 0.30146962  0.47871202  0.3151884  -0.4861505   0.10213748 -0.46538484
 -0.03540125  0.2057563   0.5080718   0.00606942 -0.20262845  0.41470525
  0.23243083 -0.47650027 -0.51098603 -0.27431914  0.0289266   0.13361788
 -0.21318203 -0.4590563  -0.03388743  0.12671329  0.31285724 -0.87205553
 -0.1736215  -0.46231928  0.02112862 -0.08940966 -0.04669904 -0.13610679
 -0.4941648   0.08937789 -0.28161564 -0.47980326 -0.23223089  0.03737691
 -0.56614333 -0.24798182 -0.53147715  0.1819585   0.08928169  0.00197849
 -0.3715223  -0.13387935 -0.10676519  0.21515708 -0.16289233  0.17692931
  0.1822422  -0.24217623]


In [40]:
test = model.wv.most_similar(['arsenic'])

In [97]:
import pandas as pd
lda = pd.read_csv('../../../scratch/boilerplate/lda_combo_results_V2.csv')

/Users/tscott/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [102]:
lda400 = lda[lda.score>=400][lda.a_id!=lda.b_id]

/Users/tscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [119]:
lda400[lda400.a_id=='EA-2006']

,a,b,score,a_id,b_id,a_page,b_page
2384714,EA-2006--EA-2006--EA-2006_FEA-2016.pdf.txt--45,EA-2088--EA-2088--final-ea-2088-columbian-deer...,422,EA-2006,EA-2088,45,22


In [125]:
lda400_acount = lda400.groupby(['a_id']).agg(
    {
         'a_id': "count",  # get the count of networks
    }
)

In [127]:
lda400_bcount = lda400.groupby(['b_id']).agg(
    {
         'b_id': "count",  # get the count of networks
    }
)

In [ ]:
lda400_bcount.names

In [123]:
lda400_acount.append(lda400_bcount)

/Users/tscott/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,a_id,b_id
1362,2.0,NaN
11009,764.0,NaN
12953,70.0,NaN
13530,763.0,NaN
17547,21.0,NaN
...,...,...
EA-2079,NaN,100.0
EA-2087,NaN,2.0
EA-2088,NaN,1.0
EA-2097,NaN,2.0


In [79]:
import re

In [89]:
def Get_ID(ival):
    return(re.sub('_.*|--.*','',ival))

In [91]:
meta_corpus['PROJECT_ID'] = meta_corpus[0].apply(Get_ID)

In [96]:
lda400

,a,b,score
0,11009--11009--11009_13574_FSPLT3_3964416.pdf.t...,13530--13530--13530_19310_FSPLT3_3986615.pdf.t...,542
1,11009--11009--11009_13574_FSPLT3_3964416.pdf.t...,20150224--20150224_Breckenridge Ski Resort Mul...,491
2,11009--11009--11009_13574_FSPLT3_3964416.pdf.t...,20150229--20150229_StonewallVegetationProjectF...,540
3,11009--11009--11009_13574_FSPLT3_3964416.pdf.t...,20150235--20150235_CRNM_final_EIS.pdf.txt--2,542
4,11009--11009--11009_13574_FSPLT3_3964416.pdf.t...,20150235--20150235_CRNM_final_Mgmt_Plan.pdf.tx...,542
...,...,...,...
2387138,EA-2090--EA-2090--FEA-2090-Marine-Geophysical-...,EA-2090--EA-2090--FEA-2090-Marine-Geophysical-...,623
2387139,EA-2090--EA-2090--FEA-2090-Marine-Geophysical-...,EA-2090--EA-2090--FEA-2090-Marine-Geophysical-...,474
2387141,EA-2090--EA-2090--FEA-2090-Marine-Geophysical-...,EA-2090--EA-2090--FEA-2090-Marine-Geophysical-...,413
2387145,EA-2097--EA-2097--fonsi-final-ea-2097-power-gr...,EA-2097--EA-2097--fonsi-final-ea-2097-power-gr...,723


In [ ]:
re.match(lda400['a'][1])

In [45]:
model.wv.most_similar(['arsenic and lead'])

KeyError: "word 'arsenic and lead' not in vocabulary"

In [14]:
ranks = []
second_ranks = []
#for doc_id in range(len(train_corpus)):
for doc_id in range(2000):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])

In [15]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 38, 1: 18, 5: 11, 11: 9, 3: 9, 2: 7, 13: 7, 25: 6, 4: 6, 21: 6, 17: 5, 43: 5, 71: 5, 9: 5, 6: 5, 46: 5, 29: 5, 77: 5, 7: 4, 20: 4, 59: 4, 19: 4, 83: 4, 12: 4, 51: 4, 18: 4, 27: 4, 34: 4, 8: 4, 14: 4, 97: 4, 36: 4, 62: 3, 295: 3, 146: 3, 10: 3, 150: 3, 75: 3, 356: 3, 54: 3, 15: 3, 23: 3, 213: 3, 345: 3, 52: 3, 497: 3, 233: 3, 81: 3, 133: 3, 117: 3, 297: 3, 339: 3, 392: 2, 1786: 2, 3959: 2, 22: 2, 7724: 2, 745: 2, 149: 2, 1444: 2, 285: 2, 2533: 2, 42: 2, 30: 2, 2645: 2, 231: 2, 555: 2, 584: 2, 208: 2, 293: 2, 4058: 2, 1096: 2, 1005: 2, 4217: 2, 7964: 2, 35: 2, 200: 2, 728: 2, 2780: 2, 568: 2, 1141: 2, 179: 2, 108: 2, 391: 2, 66: 2, 101: 2, 191: 2, 3749: 2, 1169: 2, 41: 2, 1774: 2, 868: 2, 792: 2, 24: 2, 147: 2, 911: 2, 98: 2, 708: 2, 56: 2, 508: 2, 16: 2, 621: 2, 1468: 2, 1633: 2, 498: 2, 518: 2, 132: 2, 441: 2, 175: 2, 461: 2, 1579: 2, 125: 2, 313: 2, 49: 2, 368: 2, 85: 2, 319: 2, 215: 2, 514: 2, 105: 2, 236: 2, 1318: 2, 1791: 2, 84: 2, 19464: 2, 192: 2, 121: 2, 5621: 2, 669

In [16]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (1999): «final environmental impact statement melvin sampson hatchery yakima basin coho project bull trout bull trout are species of char related to salmon and trout that prefer cold clean water they were listed as threatened under the esa in fr and spawn and rear in the upper portions of the yakima basin bull trout use the lower mainstem as migratory corridor the yakima river core area is designated as critical habitat for bull trout fr critical habitat in the core area includes the mainstem yakima river from its confluence with the columbia river upstream to the uppermost point of bull trout distribution including most tributaries in the basin the canal and bypass are not designated critical habitat for bull trout halupka the yakima river core area is part of the mid columbia recovery unit usfws the usfws identified local bull trout populations in the yakima river core area including ahtanum creek naches river tributaries american river rattlesnake creek and crow creek rimro

In [17]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (395938): «summary resource effects from no action effects from modified proposed action aquatic species tree mortality caused by the bark beetle moderate degree of impact to rainbow brown epidemics could positively affect large and brook trout woody debris recruitment to stream may result in impacts to mountain sucker channels as trees fall colorado river cutthroat wood frog leopard tree mortality due to mountain pine beetle frog and boreal toad individuals but is not likely activity along stream courses could to result in loss of viability in the planning area reduce shading and potentially increase nor cause trend toward federal listing water temperatures botany forest activities and effects to rare plants rare plants and habitats may be affected directly as result of these activities would or indirectly by soil disturbance forest canopy continue at historic levels rare plants removal or spread of invasive species no effect may be impacted by multiple ongoing for fede

IndexError: list index out of range

In [22]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model_gensim = FT_gensim(size=100)

# build the vocabulary
model_gensim.build_vocab(corpus_file=corpus_file)

# train the model
model_gensim.train(
    corpus_file=corpus_file, epochs=model_gensim.epochs,
    total_examples=model_gensim.corpus_count, total_words=model_gensim.corpus_total_words
)

print(model_gensim)

FastText(vocab=1762, size=100, alpha=0.025)


In [28]:
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os

# Create gensim dictionary form a single tet file
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in open('sample.txt', encoding='utf-8'))

# Token to Id map
dictionary.token2id


3543

In [23]:
from gensim.models.wrappers.fasttext import FastText as FT_wrapper

# Set FastText home to the path to the FastText executable
ft_home = '/home/misha/src/fastText-0.1.0/fasttext'

# train the model
model_wrapper = FT_wrapper.train(ft_home, corpus_file)

print(model_wrapper)

FileNotFoundError: [Errno 2] No such file or directory: '/home/misha/src/fastText-0.1.0/fasttext': '/home/misha/src/fastText-0.1.0/fasttext'

In [19]:
model = FastText(size=4, window=3, min_count=1)  # instantiate
model.build_vocab(sentences=common_texts)
model.train(sentences=common_texts, total_examples=len(common_texts), epochs=10)  # train

NameError: name 'FastText' is not defined